In [1]:
print('Loading modules....')

import os, sys


import uproot
import numpy as np
import matplotlib.pyplot as plt
from qetpy.utils import lowpassfilter 
from qetpy import autocuts
import cdms
#from CDMSDataCatalog import CDMSDataCatalog
import scipy.optimize as optimize
import matplotlib as mpl
import pickle as pkl
import ROOT as rt
import seaborn as sns
import operator as op
import pandas as pd
from collections import Counter
from tqdm import tqdm
print('modules loaded...')

Loading modules....


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject
/usr/local/lib/python3.8/dist-packages/CDMSDataCatalog/CDMSDataCatalog.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in 

Welcome to JupyROOT 6.24/06
modules loaded...


In [2]:
dataset = 'v2.1.1'
detNum = 1

In [3]:
prod='CUTE_T3_GeActivation_%s_V05-00_P0.2.'%(dataset)
base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/%s/Submerged'%(prod)

if(dataset == 'prompt'):
    base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Prompt/Prompt_V04-15_P0.2.1/Submerged'
    prod = 'Prompt_V04-15_P0.2.1'


file_prefix = prod

#Only 4 series
series = ['23231219_034952','23231219_110331','23231218_223530','23231217_212512']

# 11 data series
# series = [
#     '23231220_012745','23231219_184002','23231219_110331','23231219_034952','23231218_223530',
#     '23231218_190035','23231218_152721','23231218_093255','23231217_212512','23231217_171613',
#     '23231217_135018'
#          ]

# 20 data series
# series = [
#     '23231216_013604', '23231216_043946', '23231216_100125', '23231216_115012', '23231216_145300',
#     '23231216_182937', '23231216_194929', '23231216_211119', '23231216_233807', '23231217_135018',
#     '23231217_171613', '23231217_212512', '23231218_093255', '23231218_152721', '23231218_190035',
#     '23231218_223530', '23231219_034952', '23231219_110331', '23231219_184002', '23231220_012745'
# ]

filePath = []
found_series = []
for ser in series:
    
    fp = '%s/%s_%s.root'%(base,file_prefix,ser)
    if(os.path.exists(fp)):
        print(ser,' : ',fp)
        filePath.append(fp)
        found_series.append(ser)
    else:
        print('not found: ',fp)
series = found_series
print('Found series = ',series)
print('Number of found series = ',len(series))


23231219_034952  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2./Submerged/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2._23231219_034952.root
23231219_110331  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2./Submerged/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2._23231219_110331.root
23231218_223530  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2./Submerged/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2._23231218_223530.root
23231217_212512  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2./Submerged/CUTE_T3_GeActivation_v2.1.1_V05-00_P0.2._23231217_212512.root
Found series =  ['23231219_034952', '23231219_110331', '23231218_223530', '23231217_212512']
Number of found series =  4


In [4]:
print('Loading RQ and Event Tree...')
RQ=[]
for i,ser in enumerate(series):
    RQ.append(uproot.open(filePath[i])['rqDir/zip%d'%(detNum)]) 

eventTree=[]
for i,ser in enumerate(series):
    eventTree.append(uproot.open(filePath[i])['rqDir/eventTree']) 

print('done..')


print('Loading OF spefic data....')
#Load RQs from processed files
channels=['PT','PAS1','PBS1','PCS1','PDS1','PES1','PFS1','PAS2','PBS2','PCS2','PDS2','PES2','PFS2']

OFchiq={}
OFamps={}
OFdelay={}
OFamps0={}
OFnorms={}
OFbs={}

for i,key in enumerate(series):
    tc={}
    ta={}
    td={}
    ta0={}
    tn = {}
    tbs = {}
    for c in channels:
        if(detNum == '3' and c == 'PAS1'):
            print('\tSkipping Z3-PAS1...')
            continue
        tc[c]=(RQ[i][c+"OFchisq"].array(library='np'))
        ta[c]=(RQ[i][c+"OFamps"].array(library='np'))
        ta0[c]=(RQ[i][c+"OFamps0"].array(library='np'))
        td[c]=(RQ[i][c+"OFdelay"].array(library='np'))
        
        # tc[c]=(RQ[i].array(c+"OFchisq"))
        # ta[c]=(RQ[i].array(c+"OFamps"))
        # ta0[c]=(RQ[i].array(c+"OFamps0"))
        # td[c]=(RQ[i].array(c+"OFdelay"))
        if(c!='PT'):
            tn[c] = (RQ[i][c+"norm"].array(library='np'))
            tbs[c] = (RQ[i][c+"bs"].array(library='np'))
            # tn[c] = (RQ[i].array(c+"norm"))
            # tbs[c] = (RQ[i].array(c+"bs"))
            
    OFchiq[key]=tc
    OFamps[key]=ta
    OFamps0[key]=ta0
    OFdelay[key]=td
    OFnorms[key]=tn
    OFbs[key]=tbs
    print('%s loaded'%(key))
    
print('done!')

print(' Loading Event specific data....')


TriggerTime=[]
TriggerType=[]
EventNumber=[]
TrigDet=[]
TriggerMask=[]
EventTime=[]

R2 = {} # Radial parameter for S2
R1 = {} # Radial parameter for S1
amp_diff = {}  
amp_diff_abs = {}
for i,key in enumerate(series):
    TriggerTime.append(eventTree[i]["TriggerTime"].array())
    TriggerType.append(eventTree[i]["TriggerType"].array())
    EventNumber.append(eventTree[i]["EventNumber"].array())
    TrigDet.append(eventTree[i]["TriggerDetectorNum"].array())
    TriggerMask.append(eventTree[i]["TriggerMask"].array())
    EventTime.append(eventTree[i]["EventTime"].array())
    
    # TriggerTime.append(eventTree[i].array("TriggerTime"))
    # TriggerType.append(eventTree[i].array("TriggerType"))
    # EventNumber.append(eventTree[i].array("EventNumber"))
    # TrigDet.append(eventTree[i].array("TriggerDetectorNum"))
    # TriggerMask.append(eventTree[i].array("TriggerMask"))
    # EventTime.append(eventTree[i].array("EventTime"))
    R2[key] = OFdelay[key]['PFS2'] - OFdelay[key]['PAS2']
    R1[key] = OFdelay[key]['PFS1'] - OFdelay[key]['PAS1']
    amp_diff[key] = (OFamps[key]['PFS2'] - OFamps[key]['PAS2'])/(OFamps[key]['PFS2'] + OFamps[key]['PAS2'])
    amp_diff_abs[key] = (OFamps[key]['PFS2'] - OFamps[key]['PAS2'])
    print('%s loaded'%(key))

print('Done!')

Loading RQ and Event Tree...
done..
Loading OF spefic data....
23231219_034952 loaded
23231219_110331 loaded
23231218_223530 loaded
23231217_212512 loaded
done!
 Loading Event specific data....
23231219_034952 loaded
23231219_110331 loaded
23231218_223530 loaded
23231217_212512 loaded
Done!


In [5]:
chan = 'PT'
amps = {}

for c in channels:
    if(c == 'PT'):
        continue
    temp = []
    for idx, key in enumerate(series):
        mask = (TriggerType[idx] == 1) & (TrigDet[idx] == detNum) & (OFamps[key][chan] > 3e-6) & (OFamps[key][chan] < 4.2e-6) & (OFchiq[key][chan] < 5e5) & (abs(OFdelay[key][chan])*1e3 < 20)
        temp = np.append(temp, OFamps[key][c][mask])
        #print(series,' : ',EventNumber[idx][mask])
    amps[c] = temp
    print('%s loaded..: %d'%(c, len(amps[c])))



PAS1 loaded..: 143
PBS1 loaded..: 143
PCS1 loaded..: 143
PDS1 loaded..: 143
PES1 loaded..: 143
PFS1 loaded..: 143
PAS2 loaded..: 143
PBS2 loaded..: 143
PCS2 loaded..: 143
PDS2 loaded..: 143
PES2 loaded..: 143
PFS2 loaded..: 143


In [6]:
def grad(func,var,jump_var=0.001):
    """
    The gradient operator, self-explanatory.
    """
    func_now = func(*var)
    func_change_list=[0]*len(var)
    var_change_list=[0]*len(var)
    
    for i,x in enumerate(var):
        var_changed=var*1;
        var_changed[i]=(1+jump_var)*x
        #print(var)
        func_change_list[i]=func(*var_changed)-func_now
        var_change_list[i]=jump_var*x
        
    grad_list=[0]*len(var)
    for i in range(len(func_change_list)):
        grad_list[i]=func_change_list[i]/var_change_list[i]
        
    return grad_list
    
def grad_descent(func,var_start,rate,max_steps,mode="min"):
    """
    Optimizes a function with gradient descent.
    func: The function to minimize.
    var_start: A list of the initial conditions. A list of floats.
    rate: The learning rate. A list of floats. Adjust if you get bad results.
    max_steps: The maximum number of times to loop through.
    mode: "max" or "min" (default min); whether to maximize or mnimize func.
    """
    for v in var_start:
        if v==0:
            print("ERROR: Don't start the var_start at 0!")
            return -1
    switch=1
    if mode=="min":
        switch=-1
    try:
        x=func(*var_start)
    except:
        print("ERROR: Size of input arguments to function and starting variables must be the same!")
        return -1
    if len(var_start)!=len(rate):
        print("ERROR: Size of var_start must match size of rate.")
        return -1
    best_var=var_start*1
    for i in range(max_steps):
        grad_best=grad(func,best_var)
        for i,i_grad in enumerate(grad_best):
            best_var[i]+=rate[i]*i_grad*switch
    return best_var,func(*best_var)

In [9]:
def foo(a, b, c, d, e, f):    
    temp = rt.TH1F('temp', 'temp', 200, 1.0, 4.5)
    for i in range(len(amps['PAS1'])):
        s = a*amps['PAS1'][i] + b*amps['PBS1'][i] + c*amps['PCS1'][i] + d*amps['PDS1'][i] + e*amps['PES1'][i] + f*amps['PFS1'][i]
        temp.Fill(s)
    if(temp.GetMean() == 0 or temp.GetRMS() == 0 or temp.GetBinContent(0) > 0.):
        return(999.9)
    else:
        return(temp.GetRMS()/temp.GetMean())


print(grad_descent(foo,[1,1,1,1,1,1],[0.1,0.1,0.1,0.1,0.1,0.1],100,mode="max"))






([1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 999.9)
